### **Imports**

In [1]:
from file_io import load as load_image
from joblib import load
import numpy as np
import pandas as pd

### **Auxiliary Function**

_ Inputs_

* __pred -__ the  binary segmentation;
* __gt -__ the ground truth; 
* __mask -__ the binary mask; 
* __ prob -__ the probability map.

_ Outputs _

* __sen -__ Sensibility;
* __spec -__ Specificity; 
* __acc -__ Accuracy; 
* __ auc -__ Area Under the ROC curve;
* __ mcc -__ Matthews Correlation Coefficient.

In [2]:
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef

def perform_metrics (pred, gt, mask, prob):
    
    # Suppressing background regions.
    pred = pred[mask>0]
    gt = gt[mask>0]
    prob = prob[mask>0]

    # Building confusion matrix.
    c_matrix = confusion_matrix(gt, pred)
    
    # Calculating ratios.
    tn = c_matrix[0,0]
    tp = c_matrix[1,1]
    fn = c_matrix[1,0]
    fp = c_matrix[0,1]
    
    # Finding the metrics.
    sen  = tp/(tp + fn)
    spec = tn/(tn + fp)
    acc  = (tp + tn)/(tp + tn + fp + fn)
    auc  = roc_auc_score (gt, prob)
    mcc  = matthews_corrcoef (gt, pred)
    
    
    return sen, spec, acc, auc, mcc

### **Loading**

In [3]:
test_images = range(1,21)
r_width, r_height = 565, 584

In [4]:
gts = np.zeros((len(test_images), r_height, r_width))
masks = np.zeros ((len(test_images), r_height, r_width))
segs = np.zeros((len(test_images), r_height, r_width))
probs = np.zeros((len(test_images), r_height, r_width))

In order for you to run the cell below, please download the gts and masks from [DRIVE's website](https://www.isi.uu.nl/Research/Databases/DRIVE/) and change the paths accordingly.

In [5]:
#Change these paths
gts_path = '../resources/gts/DRIVE/'
masks_path = '../resources/masks/DRIVE/'

for i, image in enumerate(test_images):
    gt = load_image(gts_path + '%02d_manual1.gif' %image, normalize = True)
    mask = load_image(masks_path + '%02d_test_mask.gif' %image)
    seg = load_image('../resources/binary_segmentations/DRIVE/seg_%04d.png' %image, normalize = True)
    prob = load('../resources/probability_maps/DRIVE/prob_%04d.npy' %image)[1]
    
    gts[i] = gt
    masks [i] = mask
    segs[i] = seg
    probs[i] = prob

### **Calculating metrics**

In [6]:
sen_list, spec_list, acc_list, auc_list, mcc_list = [], [], [], [], []

for i in range (len(gts)):
    sen, spec, acc, auc, mcc = perform_metrics (segs[i], gts[i], masks[i], probs[i])
    sen_list.append(sen)
    spec_list.append(spec)
    acc_list.append(acc)
    auc_list.append(auc)
    mcc_list.append(mcc)

### **Storing the data**

In [7]:
data = {'Image' : list(range(1,21)), 'Sensitivity': sen_list, 'Specificity': spec_list, 
        'Accuracy': acc_list, 'AUC': auc_list, 'MCC': mcc_list}

df = pd.DataFrame(data = data)
df = df[['Image', 'Sensitivity','Specificity','Accuracy', 'AUC', 'MCC']]
df.set_index('Image', inplace = True);

### **Image-level results**

In [8]:
df

,Sensitivity,Specificity,Accuracy,AUC,MCC
Image,,,,,
1,0.879165,0.968076,0.956422,0.986409,0.816832
2,0.848293,0.978392,0.958900,0.987261,0.836836
3,0.762847,0.981555,0.949691,0.978555,0.788990
4,0.793692,0.984156,0.958761,0.978337,0.814988
5,0.746262,0.986031,0.953494,0.975649,0.791107
6,0.731163,0.987021,0.950914,0.975822,0.785746
7,0.754545,0.987141,0.956346,0.978605,0.800041
8,0.736428,0.985686,0.954357,0.977920,0.780624
9,0.725355,0.989523,0.958495,0.977727,0.787035


### **Average Results**

In [9]:
df.mean()

Sensitivity    0.803930
Specificity    0.980350
Accuracy       0.957630
AUC            0.982132
MCC            0.805388
dtype: float64